In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("fake_news_train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
##drop nan values
df.dropna(inplace=True)

In [5]:
df.shape

(18285, 5)

In [6]:
x = df.drop("label", axis =1)

In [7]:
y = df['label']

In [8]:
x.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.3.1'

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
### Vocabulary size
voc_size = 5000

# Onehot Representation

In [14]:
messages = x.copy()

In [15]:
messages.reset_index(inplace=True)

In [16]:
messages["title"][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [17]:
import re
import nltk
from nltk.corpus import stopwords

In [18]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\varma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
##Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
    review = re.sub("[^a-zA-Z]"," ", messages["title"][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(words) for words in review if not words in set(stopwords.words("english"))]
    review = " ".join(review)
    corpus.append(review)

In [20]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [21]:
messages["title"][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [22]:
Onehot_repr = [one_hot(words,voc_size) for words in corpus]
Onehot_repr

[[726, 2657, 2347, 385, 1104, 4491, 70, 3417, 4727, 1718],
 [1004, 3841, 4086, 815, 4530, 171, 1601],
 [534, 3500, 2215, 260],
 [3213, 3399, 1418, 3854, 2788, 269],
 [19, 4530, 4298, 1095, 219, 3477, 4530, 3843, 3147, 3746],
 [4637,
  1443,
  462,
  243,
  3101,
  4707,
  530,
  1668,
  480,
  177,
  1156,
  3403,
  1434,
  459,
  1601],
 [396, 2946, 13, 1777, 4460, 3220, 3244, 1868, 1785, 660, 3347],
 [1325, 3670, 2858, 104, 331, 1679, 4707, 1808, 1785, 660, 3347],
 [4289, 2850, 2819, 2899, 3428, 702, 2727, 1200, 4707, 4319],
 [2989, 862, 2279, 712, 348, 4458, 2481, 4489],
 [3319, 4198, 2771, 3151, 285, 2729, 1372, 3433, 1986, 2352, 2935],
 [3854, 4618, 1104, 702, 4707, 331],
 [1133, 2333, 3773, 4622, 1055, 223, 3041, 724, 2021],
 [2927, 1058, 2723, 3677, 666, 4728, 4603, 1785, 660, 3347],
 [3583, 1663, 4581, 1630, 447, 1785, 660, 3347],
 [1628, 4891, 983, 2500, 4518, 156, 3670, 1230, 3443, 2107],
 [2295, 1856, 3841],
 [2650, 1664, 1762, 4671, 4707, 1287, 2028, 1601],
 [3499, 761, 408

# Embedding representatiion

In [23]:
sent_length = 20
embedded_docs = pad_sequences(Onehot_repr, padding="pre", maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 3417, 4727, 1718],
       [   0,    0,    0, ..., 4530,  171, 1601],
       [   0,    0,    0, ..., 3500, 2215,  260],
       ...,
       [   0,    0,    0, ..., 1785,  660, 3347],
       [   0,    0,    0, ...,  170, 4131, 2498],
       [   0,    0,    0, ..., 2442, 1141,  355]])

In [24]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  726,
       2657, 2347,  385, 1104, 4491,   70, 3417, 4727, 1718])

In [25]:
##this is the input given to convert to number of features

In [26]:
##creating the model
embedding_vector_features = 40 ##dimensions
model = Sequential()
##now send it to embedding layer 
model.add(Embedding(voc_size, embedding_vector_features, input_length= sent_length))
model.add(LSTM(100)) ##1 layer with 100 neurons
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [27]:
##store in other variables to test the model

In [28]:
import numpy as np
x_final = np.array(embedded_docs)#embedded_docs is the features to test the model
y_final = np.array(y)

In [29]:
x_final.shape, y_final.shape

((18285, 20), (18285,))

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

# Model Training

In [31]:
###finally training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 51ms/step - loss: 0.3365 - accuracy: 0.8428 - val_loss: 0.1956 - val_accuracy: 0.9183
Epoch 2/10
192/192 [==============================] - 7s 39ms/step - loss: 0.1372 - accuracy: 0.9464 - val_loss: 0.1960 - val_accuracy: 0.9208
Epoch 3/10
192/192 [==============================] - 9s 46ms/step - loss: 0.0874 - accuracy: 0.9672 - val_loss: 0.2169 - val_accuracy: 0.9157
Epoch 4/10
192/192 [==============================] - 9s 46ms/step - loss: 0.0522 - accuracy: 0.9817 - val_loss: 0.2538 - val_accuracy: 0.9075
Epoch 5/10
192/192 [==============================] - 9s 44ms/step - loss: 0.0303 - accuracy: 0.9921 - val_loss: 0.3216 - val_accuracy: 0.9150
Epoch 6/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0167 - accuracy: 0.9951 - val_loss: 0.4390 - val_accuracy: 0.9077
Epoch 7/10
192/192 [==============================] - 10s 51ms/step - loss: 0.0108 - accuracy: 0.9977 - val_loss: 0.4602 - val_accuracy: 0.91

# Performance metrics

In [32]:
from sklearn import metrics

In [33]:
from sklearn.metrics import confusion_matrix

In [34]:
y_pred = model.predict_classes(X_test)
y_pred

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [35]:
confusion_matrix(y_test,y_pred)

array([[3103,  316],
       [ 222, 2394]], dtype=int64)

In [36]:
y_test

array([1, 0, 0, ..., 0, 1, 1], dtype=int64)

In [37]:
y_pred.shape, y_test.shape

((6035, 1), (6035,))

In [38]:
x_test

NameError: name 'x_test' is not defined

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
x_final

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)